## Imports

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import math
import re

#from bs4 import BeautifulSoup
#from nltk.corpus import stopwords
#from gensim.models import word2vec, Word2Vec

from util.word2vec_as_MF import Word2vecMF
from util.functions import *

import time
dimension = 100
regularization =0.0

## Read and preprocess enwik9

In [2]:
real_sentences = load('data/x1')

Parsing sentences from training set


# Gensim

In [3]:
'''%%time
skip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'''

'%%time\nskip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'

In [4]:
'''skip.get_latest_training_loss()'''

'skip.get_latest_training_loss()'

## Train ro_sgns model starting from SVD of SPPMI

In [5]:
# Create word2vec as matrix factorization model
model_enwik = Word2vecMF()
model_enwik.data_to_matrices(real_sentences, dimension, 5, 'enwik-200/tmp.npz')
#model_enwik.data_to_matrices(real_sentences, dimension, 5, False)

In [6]:
# If the model has been already created, load it from file
'''model_enwik.load_matrices(from_file='enwik-200/tmp.npz')'''

"model_enwik.load_matrices(from_file='enwik-200/matrices1.npz')"

In [7]:
#C0, W0, step_size = BFGD_init(model_enwik, dimension=dimension, reg=regularization)
C0, W0 = SPPMI_init(model_enwik, dimension=dimension)

/home/yerong/Documents/BFGD-sgns/util/functions.py:118: RuntimeWarning: divide by zero encountered in log
  SPPMI = np.maximum(np.log(model.D) - np.log(model.B), 0)


Initial loss (1808805.021334937, 1.0321434290463413e-13)


In [8]:
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='PS', 
               d=dimension,
               eta = 5e-5,
               MAX_ITER=10,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'PS\', \n               d=dimension,\n               eta = 5e-5,\n               MAX_ITER=10,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [9]:
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='AM', 
               d=dimension,
               eta = 5e-6,
               lbd = 1.0,
               MAX_ITER=189000,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'AM\', \n               d=dimension,\n               eta = 5e-6,\n               lbd = 1.0,\n               MAX_ITER=189000,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [10]:
start_time = time.time()
opt_experiment(model_enwik,
               mode='PS',
               d=dimension,
               eta=5e-5,
               #eta = step_size,
               reg = regularization,
               MAX_ITER=1000,
               from_iter=0,
               start_from='9PMM',
               itv_print=1,
               itv_save=10,
               init=(True, C0, W0), display=True)
print("--- %s seconds ---" % (time.time() - start_time))

enwik-200/PSiter_from9PMM_dim100_step5e-05_0.0
Iter #: 1 loss (1808805.021334937, 7.573699547840907e-14)
Iter #: 2 loss (1807690.3576354615, 7.338592998494247e-14)
Iter #: 3 loss (1806656.4267292053, 7.591659688705824e-14)
Iter #: 4 loss (1805556.4983759832, 7.671899015689629e-14)
Iter #: 5 loss (1804207.835323143, 7.490444611334936e-14)
Iter #: 6 loss (1802360.85175471, 7.79847083708971e-14)
Iter #: 7 loss (1799674.7275180353, 7.678122855795961e-14)
Iter #: 8 loss (1795726.23614117, 7.448263885524365e-14)
Iter #: 9 loss (1790097.8460254797, 7.650688490063054e-14)
Iter #: 10 loss (1782567.4806659597, 7.327179575088283e-14)
Iter #: 11 loss (1773305.2077464296, 7.359438461464365e-14)
Iter #: 12 loss (1762876.0543610991, 7.331012600596454e-14)
Iter #: 13 loss (1751994.284832052, 7.617297849470928e-14)
Iter #: 14 loss (1741246.514590741, 7.854633625644792e-14)
Iter #: 15 loss (1730982.6557167668, 7.632241086527761e-14)
Iter #: 16 loss (1721353.5246006409, 7.404192665236434e-14)
Iter #: 17 

KeyboardInterrupt: 

In [ ]:
'''model = Word2Vec(real_sentences, size=200, window=5, min_count=5, workers=4)
fname = 'original'
model.save(fname)
model = Word2Vec.load(fname)  # you can continue training with the loaded model!'''